Create data source

In [8]:
import great_expectations as gx

context = gx.get_context()

# remove existing data source, if needed
context.sources.delete_pandas_filesystem("temperature")

# create a new data source
datasource_name = "temperature"
datasource = context.sources.add_pandas_filesystem(
    name=datasource_name,
    base_directory="../data/preprocessed/temp"
)

# add new data asset to the data source
data_asset_name = "temperature_data"
data_asset = datasource.add_csv_asset(
    name=data_asset_name,
    batching_regex=r"(?P<station>.*)\.csv"
)

# list existing data sources
print(context.datasources)

{'temperature': PandasFilesystemDatasource(type='pandas_filesystem', name='temperature', id=None, assets=[CSVAsset(name='temperature_data', type='csv', id=None, order_by=[], batch_metadata={}, batching_regex=re.compile('(?P<station>.*)\\.csv'), connect_options={}, splitter=None, sep=None, delimiter=None, header='infer', names=None, index_col=None, usecols=None, dtype=None, engine=None, true_values=None, false_values=None, skipinitialspace=False, skiprows=None, skipfooter=0, nrows=None, na_values=None, keep_default_na=True, na_filter=True, verbose=None, skip_blank_lines=True, parse_dates=None, infer_datetime_format=None, keep_date_col=None, date_format=None, dayfirst=False, cache_dates=True, iterator=False, chunksize=None, compression='infer', thousands=None, decimal='.', lineterminator=None, quotechar='"', quoting=0, doublequote=True, escapechar=None, comment=None, encoding=None, encoding_errors='strict', dialect=None, on_bad_lines='error', delim_whitespace=None, low_memory=True, memor

Querying batch data source from data context

In [9]:
from pprint import pprint

# build batch request
batch_request = data_asset.build_batch_request()

# use keyword argument to pass the batch_request
batches = context.get_batch_list(batch_request=batch_request)

# print batch identifiers
pprint([batch.batch_request.options for batch in batches])

[{'path': 'BOVEC.csv', 'station': 'BOVEC'},
 {'path': 'CELJE.csv', 'station': 'CELJE'},
 {'path': 'CRNOMELJ.csv', 'station': 'CRNOMELJ'},
 {'path': 'LENDAVA.csv', 'station': 'LENDAVA'},
 {'path': 'MEZICA.csv', 'station': 'MEZICA'},
 {'path': 'PTUJ.csv', 'station': 'PTUJ'},
 {'path': 'ROGLA.csv', 'station': 'ROGLA'}]


Create expectations suite - containing rules for validating data

In [10]:
# add or update expectation suite
expectation_suite_name = "temperature_suite"
expectation_suite = context.add_or_update_expectation_suite(
    expectation_suite_name="temperature_suite"
)

Create automatic expectation suite (validation rules)

In [11]:
# generate expectation suite, based on which we will automatically create validation rules
asset = context.get_datasource(datasource_name).get_asset(data_asset_name)

# create a validator from the asset using a batch request, which will load data from data source
batch_request = asset.build_batch_request()
validator = context.get_validator(
    batch_request=batch_request,
    expectation_suite_name=expectation_suite_name
)

# run data assistant with the validator to create validation rules
exclude_column_names = []
data_assistant_result = context.assistants.onboarding.run(
    validator=validator,
    exclude_column_names=exclude_column_names
)

# save expectation suite
expectation_suite = data_assistant_result.get_expectation_suite()
context.save_expectation_suite(
    expectation_suite=expectation_suite,
    expectation_suite_name="temperature_suite"
)

Generating Expectations:   0%|          | 0/8 [00:00<?, ?it/s]

Profiling Dataset:         0%|          | 0/1 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/7 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/14 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/14 [00:00<?, ?it/s]

...

Calculating Metrics:   0%|          | 0/28 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/28 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/28 [00:00<?, ?it/s]

Calculating Metrics:   0%|          | 0/49 [00:00<?, ?it/s]

'g:\\Other computers\\Home Desktop\\Google Drive\\FERI\\MAGISTERIJ\\4. semester\\Inzenirstvo Inteligentnih Sistemov\\vaje\\intelligent_system\\gx\\expectations/temperature_suite.json'

Create checkpoint

- it needs to be connected to the expectation suite
- allows us to execute validations at every run and to view results

In [12]:
# create checkpoint
checkpoint_name = "temperature_checkpoint"
batch_request = asset.build_batch_request()

checkpoint = context.add_or_update_checkpoint(
    name=checkpoint_name,
    validations=[
        {
            "batch_request": batch_request,
            "expectation_suite_name": expectation_suite_name
        }
    ],
)

Run checkpoint and build data docs

In [13]:
# run checkpoint
checkpoint_result = checkpoint.run()

# build data docs
site = context.build_data_docs()
context.open_data_docs()

Calculating Metrics:   0%|          | 0/331 [00:00<?, ?it/s]